In [17]:
import numpy as np

## Simulate PCR

We begin with approximately 100-500 thousand DMS modified molecules. Right now, we can DMS modify molecules of length 1000-2000nt. 

We perform PCR to produce approximately 100 billion molecules (from the original 100-500). Currently looking for sophisticated tools for simulating the PCR process, but in the meanwhile, using a naive sample and duplicate algorithm.



In [24]:
def naive_PCR(molecules, max_molecules):
    '''
    Naively simulate PCR by randomly sampling a molecule from the list of molecules and duplicating this molecule.
    Repeat this naive amplification until we have the desired number of molecules.
    @param: molecules - numpy array of molecules (each molecule is a list)
    @param: max_molecules - desired total number of molecules after PCR
    
    @param: molecules -  a numpy array of all the molecules after PCR'''
    

    indices = [i for i in range(len(molecules))]
    while len(indices)<max_molecules:
        dup = np.random.choice(indices)
        indices.append(dup)

    indices.sort()
    return molecules[indices]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 29, 37, 7, 3, 44, 41, 29, 17, 16, 9, 7, 37, 19, 19, 37, 30, 32, 19, 42, 21, 37, 42, 11, 11, 7, 37, 37, 16, 39, 5, 21, 44, 21, 9, 42, 26, 28, 27, 30, 43, 11, 41, 11, 6, 7, 21, 4, 7, 42, 6, 30, 5, 16, 30, 4]


In [25]:
indices.sort()

In [27]:
molecules[indices]

array([ 0.78719312,  0.87596597,  0.87861641,  0.72939428,  0.72939428,
        0.83512515,  0.83512515,  0.83512515,  0.75222423,  0.75222423,
        0.75222423,  0.20104062,  0.20104062,  0.20104062,  0.30549686,
        0.30549686,  0.30549686,  0.30549686,  0.30549686,  0.30549686,
        0.79100941,  0.53529002,  0.53529002,  0.53529002,  0.7327094 ,
        0.25296627,  0.25296627,  0.25296627,  0.25296627,  0.25296627,
        0.89737663,  0.10386767,  0.64234535,  0.98670725,  0.70794128,
        0.70794128,  0.70794128,  0.70794128,  0.89129383,  0.89129383,
        0.22480221,  0.7663735 ,  0.7663735 ,  0.7663735 ,  0.7663735 ,
        0.77303575,  0.83417898,  0.83417898,  0.83417898,  0.83417898,
        0.83417898,  0.01240468,  0.68502602,  0.41472956,  0.46782353,
        0.44595178,  0.44595178,  0.44899798,  0.44899798,  0.08864527,
        0.08864527,  0.39176231,  0.39176231,  0.39176231,  0.44781307,
        0.44781307,  0.44781307,  0.44781307,  0.44781307,  0.57

In [28]:
indices

[0,
 1,
 2,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 9,
 9,
 9,
 10,
 11,
 11,
 11,
 11,
 11,
 12,
 13,
 14,
 15,
 16,
 16,
 16,
 16,
 17,
 17,
 18,
 19,
 19,
 19,
 19,
 20,
 21,
 21,
 21,
 21,
 21,
 22,
 23,
 24,
 25,
 26,
 26,
 27,
 27,
 28,
 28,
 29,
 29,
 29,
 30,
 30,
 30,
 30,
 30,
 31,
 32,
 32,
 33,
 34,
 35,
 36,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 39,
 39,
 40,
 41,
 41,
 41,
 42,
 42,
 42,
 42,
 42,
 43,
 43,
 44,
 44,
 44]